In [9]:
!pip install chromadb

In [10]:
from PIL import Image
import torch
import torchvision.transforms as transforms
from chromadb.config import Settings
from chromadb import Client

In [11]:
# Step 1: Define image preprocessing and embedding extraction
class ImageEmbedder:
    def __init__(self):
        # Use a pre-trained model (e.g., ResNet) for embedding extraction
        self.model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
        self.model.fc = torch.nn.Identity()  # Remove the final classification layer
        self.model.eval()

        # Define image preprocessing
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

    def extract_embedding(self, image_path):
        image = Image.open(image_path).convert("RGB")
        image_tensor = self.transform(image).unsqueeze(0)
        with torch.no_grad():
            embedding = self.model(image_tensor).squeeze().numpy()
        return embedding



In [12]:
# Step 2: Initialize ChromaDB client
client = Client(Settings())
collection = client.get_or_create_collection(name="image_collection")

In [14]:
# Step 3: Add images to the vector database
image_embedder = ImageEmbedder()
image_paths = ["/content/sample_data/image1.jpg", "/content/sample_data/image2.jpg", "/content/sample_data/image3.jpg"]  # Replace with your image paths

for idx, image_path in enumerate(image_paths):
    embedding = image_embedder.extract_embedding(image_path)
    # The 'add' method expects 'documents', 'ids', and 'metadatas' as arguments.
    # We're adapting the code to provide the data in this format.
    collection.add(
        documents=[str(embedding)],  # Convert embedding to string
        ids=[f"img_{idx}"],
        metadatas=[{"path": image_path}]
    )

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:00<00:00, 103MiB/s]


In [19]:
# Step 3: Add images to the vector database
image_embedder = ImageEmbedder()
image_paths = ["/content/sample_data/image1.jpg", "/content/sample_data/image2.jpg", "/content/sample_data/image3.jpg"]  # Replace with your image paths

# Delete the existing collection to ensure consistent dimensionality
client.delete_collection(name="image_collection")
collection = client.get_or_create_collection(name="image_collection") # Recreate the collection

for idx, image_path in enumerate(image_paths):
    embedding = image_embedder.extract_embedding(image_path)
    # Instead of converting the embedding to a string, pass it directly as a list
    collection.add(
        embeddings=[embedding.tolist()],  # Pass embedding as a list
        ids=[f"img_{idx}"],
        metadatas=[{"path": image_path}]
    )

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [22]:

# Step 4: Perform similarity search
def search_similar_images(query_image_path, top_k=3):
    query_embedding = image_embedder.extract_embedding(query_image_path)
    results = collection.query(query_embeddings=[query_embedding], n_results=top_k)
    # Accessing the first element of each metadata list, assuming it's a dictionary
    for result in results["metadatas"]:
        print("Similar Image:", result[0]["path"]) # Accessing the first element of each metadata list

In [24]:
# Test similarity search
query_image = "/content/sample_data/sample-image.jpg"  # Replace with your query image path
search_similar_images(query_image, top_k=3)

Similar Image: /content/sample_data/image3.jpg
